# River Network

In [ ]:
import geopandas as gpd
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import contextily as ctx
import os

In [ ]:
import sfrmaker

In [ ]:
import flopy

In [ ]:
import basic
m = basic.load_model()

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import pyproj
import proplot as pplt

In [ ]:
import rioxarray  # noqa: F401
import xarray as xr

Now, let's use [WaterData](https://labs.waterdata.usgs.gov/geoserver/web/) service to get the headwater catchments for this basin:

In [ ]:
lines = sfrmaker.Lines.from_nhdplus_hr(r"C:\GIS\shapefiles\NHD\NHDPLUS_H_1801_HU4_GDB\NHDPLUS_H_1801_HU4_GDB.gdb")

## if running again, check to make sure of the following
- that the streamflow ending at TOP of SWR does not get routed to next SFR via SFR. this has lead to double counting of flow below SWR
- make sure that the number of days in tab files matches length of run 
- the re-read (-1) at end of file is copied number of days

In [ ]:
sfrdata = lines.to_sfr( model=m, model_length_units='feet')

In [ ]:
# load SWR cells to remove from processing

swr = gpd.read_file("GIS/SWR_Reaches.shp")

ib1 = m.bas6.ibound.array[0]
ib1[swr.loc[:,'i'],swr.loc[:,'j']] = 0
ib2 = m.bas6.ibound.array[1]
ib2[swr.loc[:,'i'],swr.loc[:,'j']] = 0
ib3 = m.bas6.ibound.array[2]
ib3[swr.loc[:,'i'],swr.loc[:,'j']] = 0

plt.imshow(ib1)

m.bas6.ibound = [ib1, ib2, ib3]


plt.figure()
m.bas6.ibound.plot( inactive  = True)

In [ ]:


sfrdata = lines.to_sfr( model=m, model_length_units='feet')



In [ ]:
plt.imshow(np.ma.array(m.dis.top.array, mask = m.bas6.ibound.array[0]==0), vmax = 200, vmin = 30)

In [ ]:
sfrdata.mf5names

In [ ]:
sfr_reach = sfrdata.reach_data
sfr_reach.loc[:,'width'] = sfr_reach.loc[:,'width'].where(sfr_reach.loc[:,'width']<30,30)
sfr_reach.loc[:,'model_top'] = m.dis.top.array[sfr_reach.loc[:,'i'], sfr_reach.loc[:,'j']]

sfr_reach.loc[:,'top_diff'] = sfr_reach.loc[:,'model_top'] - sfr_reach.loc[:,'strtop']

display(sfr_reach.filter(regex = 'top').describe())

plt.scatter(sfr_reach.loc[:,'model_top'],sfr_reach.loc[:,'strtop'])
plt.figure()
plt.scatter(sfr_reach.loc[:,'model_top'],sfr_reach.loc[:,'top_diff'])

# update sfr strtop to top of model top

In [ ]:
# sfrdata.reach_data.loc[:,'strtop'] = m.dis.top.array[sfrdata.reach_data.loc[:,'i'], sfrdata.reach_data.loc[:,'j']]
# sfrdata.reach_data.loc[:,'strhc1'] = sfr_info['hcond1'] # hyd k =0.4m/day from zhang
# sfrdata.reach_data.loc[:,'strhc1'] = 1e-15
# # sfrdata.reach_data.loc[:,'hcond1'] = 0.000015 # hyd k =0.4m/day from zhang
# sfr_flopy.reach_data['strthick'] = 10.

# info for tabfiles

In [ ]:

info, swr_info,sfr_info, riv_keys = basic.load_params(run_name='pest2014trans')
riv_keys

# get length of files
for k in riv_keys:

    x =  riv_keys[k]
    with open(os.path.join(m.model_ws,'ref', x['file']), 'r') as temp:
        num = temp.readlines()
    x['numval'] = len(num)
    riv_keys[k] =x

tabfiles_dict = {riv_keys[key]['iseg']:{'numval':riv_keys[key]['numval'],
                                       'inuit':riv_keys[key]['inuit']} 
                 for key in riv_keys.keys()}
tabfiles = True
tabfiles_dict

In [ ]:
dataset5 = sfrdata.modflow_sfr2.dataset_5
for x in  np.arange(1,m.dis.nper):
    dataset5[x]=[-1,0,0] 

In [ ]:
filename = 'RR_2022/sfr_only.sfr'

In [ ]:
sfr_flopy = sfrdata.modflow_sfr2

In [ ]:
icalc = 2

In [ ]:
if icalc ==1:
    sfr_flopy.segment_data[0]['width2'] = np.where(sfr_flopy.segment_data[0]['width2']<30,
                                                   sfr_flopy.segment_data[0]['width2'], 30 )
elif icalc ==2:
    # sfr_flopy.segment_data[0]['width2']
    sfr_flopy.segment_data[0]['icalc'] = 2
    
    sfr_flopy.segment_data[0]['roughbk'] = 0.037

In [ ]:
pd.DataFrame(sfr_flopy.segment_data[0]).columns

In [ ]:
sfr_flopy.dataset_5 = dataset5
sfr_flopy.tabfiles = tabfiles
sfr_flopy.numtab = 3
sfr_flopy.tabfiles_dict = tabfiles_dict
sfr_flopy.istcb2 = 746
sfr_flopy.ipakcb = 1
sfr_flopy.const = 1.486

In [ ]:
sfr_flopy.maxval = 1095

In [ ]:
sfr_flopy.options= flopy.modflow.mfsfr2.OptionBlock('tabfiles 3 365\ntransroute\n', flopy.modflow.ModflowSfr2)
# sfr_flopy.options= flopy.modflow.mfsfr2.OptionBlock('tabfiles 3 365', flopy.modflow.ModflowSfr2)

In [ ]:
sfr_flopy.options

In [ ]:
sfr_flopy.reach_data['strtop'] = m.dis.top.array[sfr_flopy.reach_data['i'], sfr_flopy.reach_data['j']]
sfr_flopy.reach_data['strhc1'] = sfr_info['hcond1'] # hyd k =0.4m/day from zhang
# sfr_flopy.reach_data['strhc1'] = 1e-15
# sfrdata.reach_data.loc[:,'hcond1'] = 0.000015 # hyd k =0.4m/day from zhang
sfr_flopy.reach_data['strthick'] = 10.

In [ ]:
sfr_info['hcond1']

In [ ]:
pd.DataFrame(sfr_flopy.segment_data[0]).columns

In [ ]:
pd.DataFrame(sfr_flopy.reach_data).head()

In [ ]:
from io import StringIO

In [ ]:
# cross section geometry from section 98 (or 97 ??) from SWR geometry
geo = '''0.0	63.199999999999996
26.854843138249844	58.559999999999995
42.997819711460366	55.53
58.380553268229235	54.54
73.38335574147675	54.15
78.1642028035381	54.65
86.67570651494495	56.91
91.34657355283481	55.78
103.7588935940099	55.08
119.42176685959176	54.269999999999996
136.15482694333704	54.05
147.48677127115275	53.489999999999995
162.79951750498233	53.46
164.01986769830236	52.28
169.2308272737203	53.089999999999996
173.37160003861277	53.72
192.30502281487412	53.6
205.4374026310002	53.86
208.6279667253115	52.23
212.65856883664918	53.519999999999996
220.7501488107839	52.41
227.27125159109005	50.48
237.09308910196404	50.129999999999995
240.07369139428582	50.03
244.72456210984552	50.57
258.8571414888715	48.449999999999996
268.3790338300319	44.94
277.48081194892603	42.239999999999995
285.2922522600216	40.559999999999995
293.11369074091533	39.309999999999995
306.41623080319374	38.559999999999995
336.14188804643715	37.85
340.7727625558931	37.44
359.52641029550983	35.629999999999995
373.03891191108704	34.48
402.00451800913913	33.86
434.5506159238245	33.559999999999995
452.2240794116301	33.33
466.4866344274813	31.04
470.13730558191816	32.26
474.4482396420468	33.68
475.37845239256797	34.199999999999996
480.999528585266	37.3
481.30947092255474	37.75
483.4900297828913	40.76
487.19069161842543	43.16
492.76177712960526	46.75
497.0725183507133	49.55
498.63280678181485	49.69
503.4836404488846	50.19
509.5348295253918	50.94
513.0955174234158	51.26
513.7557805019808	51.269999999999996
514.5458263354556	51.35
519.9169491364388	51.87
526.6182101860251	52.51
532.9193527160802	53.19
533.2694805629903	53.23
538.3002806047947	54.879999999999995
542.7311924883978	56.449999999999996
545.3416713393804	57.489999999999995
546.2018971951618	57.769999999999996
547.3220745591356	58.1
549.9225552742004	58.089999999999996
552.3030770321495	57.699999999999996
555.02353535676	57.79
558.724197166263	58.589999999999996
562.414860845214	58.79
566.4056616946841	58.71
571.1665120606534	59.269999999999996
572.1167211327994	59.26
573.3070786229205	59.01
575.1973054525124	58.96
575.2672923946767	58.94
577.6778085570465	58.21
579.0381343224805	58.28
581.5586299756314	58.78
584.1391144232833	59.54
584.7791881555926	59.53
587.0295409942454	58.739999999999995
588.5898294223513	58.83
589.670207488491	59.12
591.100326932042	59.42
595.1811110067642	59.48
596.4514535985497	59.41
603.3926698255411	59.22
605.6832084507809	59.309999999999995
607.1135211803471	59.559999999999995
612.4944489543274	60.739999999999995
615.3050837587494	61.08
619.6658155003944	61.39
623.8765752933208	61.93
627.3972707138291	62.199999999999996
629.0475423206332	62.35
631.9481602939568	62.8'''


geo = pd.read_csv(StringIO(geo), sep="\t", names = ['x','SWR Geometry'])

ax = geo.set_index('x').plot(label = 'raw points')
print(geo.shape)
pts = [0, 20,29,34, 37, 38,49 ,  geo.shape[0]-1]

geo.loc[pts].set_index('x').rename(columns = {'SWR Geometry':"SFR Geometry"}).plot(
    ls = '-', c = 'r', ax = ax, marker = 'o')

geo = [geo.loc[pts,'x'].apply(lambda x: np.round(x,0)).values, geo.loc[pts,'SWR Geometry'].values, ]
geo

In [ ]:
geo

In [ ]:

if icalc==2:
    
    markwest_geo = pd.read_csv('GIS\Stream_GEO\markwest_creek_xsection.csv', index_col = [0])
    
    markwest_geo = [markwest_geo.values[:,0], markwest_geo.values[:,1]]
    
    # including both makes it crash...
    # mw_segments = [50, 55]
    #downstream only
    mw_segments = [55]
    # geo = [[0,1,2,3,4],[10,1,2,10,30]]
    # geo = [[0,	11.1,	22.21,	33.31,	44.41,	55.51,	66.62,	120],
    # [7.77,	3.11,	2.33,	0,	0.78,	2.33,	3.11,	10]]

    fig, ax = plt.subplots()
    ax.scatter(geo[0], geo[1])
    ax.set_title('geometry points for channel geometry')
    
    channel_geo =  {x:geo for x in
                sfr_flopy.segment_data[0].nseg}
    
    for seg in mw_segments:
        channel_geo[seg] = markwest_geo
        
    channel_geo = {0:channel_geo}
    
    
    
    sfr_flopy.channel_geometry_data = channel_geo

In [ ]:
testing = False

In [ ]:
if testing:
    print(f"writing testing file for sfr {filename.replace('sfr_','sfr_test')}")
    sfr_flopy.write_file(filename=filename.replace('sfr_',"sfr_test"))
else:
    sfr_flopy.write_file(filename=filename)

In [ ]:
import conda_scripts.arich_functions as af
import conda_scripts.utils as rg

In [ ]:
sfr_flopy.options

In [ ]:
geom = af.get_model_shp(m.modelgrid, 2226)
sfr_data_spat = pd.DataFrame(sfr_flopy.reach_data)

sfr_data_spat = gpd.GeoDataFrame(pd.merge(geom, sfr_data_spat), geometry = 'geometry', crs = 2226)

In [ ]:
mmm= sfr_data_spat.astype({'iseg':np.float64}).explore( name = 'sfr')
mmm

In [ ]:
swr = gpd.read_file(r'SFR_files\only_sfr_cells.shp')
swr.explore()

In [ ]:
swr = gpd.read_file(r'SFR_files\only_sfr_cells.shp')
mmm= sfr_data_spat.loc[~sfr_data_spat.loc[:,'node'].isin(swr.node)].explore(style_kwds  = dict(fillColor  = 'w', color = 'r'), name = 'sfr')

# swr.explore(m = mmm, style_kwds  = dict(fillColor  = 'c'), name = 'swr')
rg.folium_maps.add_layers(mmm)
mmm

In [ ]:
pd.DataFrame(sfr_flopy.segment_data[0]).width2

In [ ]:

# sfrdata.write_package(filename=filename, version='mfnwt',
#                                  tabfiles=tabfiles,
#                                 tabfiles_dict=tabfiles_dict,
#                       dataset_5= dataset5
#                                  )

with open(filename, 'a') as temp:
    for _ in range(m.dis.nper*4):
        temp.write('-1\n')


In [ ]:
sfrdata.write_shapefiles(r'SFR_files\sfr_only.shp')

In [ ]:
mod = gpd.read_file('GIS/grid.shp')

In [ ]:
outlets

In [ ]:
cells.query("name=='Russian River'").sort_values('outreach')

In [ ]:
mod_boundary = gpd.read_file("GIS/model_boundary.shp")
mod_boundary = gpd.GeoDataFrame(mod_boundary)

In [ ]:
mw = cells.explore('name')

swr.explore(color = 'black', m = mw, name = 'SWR Reaches')
# mod_boundary.explore(m=m, style_kwds = {'fill':False})


In [ ]:
# routing = gpd.read_file('SFR_files\only_sfr_routing.shp')
cells = gpd.read_file('SFR_files\only_sfr_cells.shp')
outlets = gpd.read_file('SFR_files\only_sfr_outlets.shp')
model_boundary_5070 = mod.to_crs(epsg=2226)

fig, ax = plt.subplots(figsize=(10,8))
cells.plot('name',ax = ax, zorder = 200, facecolor = 'r')
# routing.plot(ax=ax, zorder=3)
outlets.plot(ax=ax, c='red', zorder=4, label='outlets')
model_boundary_5070.plot(ax=ax, facecolor='None', 
                         edgecolor='gray',
                         zorder=1
                        ) 

LegendElement = [
    mpatches.mlines.Line2D([], [], color='red', linewidth=0., marker='o', label='sfr outlet'),
    mpatches.mlines.Line2D([], [], color='#1f77b4', label='sfr routing'),
    mpatches.Patch(facecolor='None', edgecolor='gray', label='Model Boundary\n(active area)')
]

ax.legend(handles=LegendElement, loc='upper left')


f = flopy.plot.PlotMapView(m, ax =  ax)
# f.plot_array(m.bas6.ibound[0])
f.plot_ibound(color_noflow = 'black', alpha = .1)
ctx.add_basemap(ax, crs = 2226)
plt.show()

In [ ]:
mlnew = basic.load_model()

In [ ]:
mlnew.sfr.check()

In [ ]:
mlnew.sfr.stress_period_data.df

In [ ]:
mlnew.sfr.plot()